## Modelos de Regresión

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
df = pd.read_excel('data_clusters.xlsx', index_col= 0)

In [3]:
risk_labels = {
    0: 'Riesgo Moderado',
    1: 'Riesgo Alto',
    2: 'Riesgo Bajo'
}

# Asignación de etiquetas de riesgo a la columna 'clusters'
df['cluster'] = df['cluster'].map(risk_labels)

# Imprimir el DataFrame actualizado
df

,departamento,municipio,armas_medios,fecha_hecho,genero,grupo_etario,cantidad,cluster
0,ATLANTICO,BARRANQUILLA (CT),ARMA BLANCA,2010-01-01,MASCULINO,ADULTOS,1,Riesgo Alto
1,BOYACA,DUITAMA,ARMA BLANCA,2010-01-01,FEMENINO,ADULTOS,1,Riesgo Alto
2,CAQUETA,PUERTO RICO,ARMA BLANCA,2010-01-01,MASCULINO,ADULTOS,1,Riesgo Moderado
3,CASANARE,MANÍ,ARMA BLANCA,2010-01-01,FEMENINO,ADULTOS,1,Riesgo Moderado
4,SANTAFE DE BOGOTA D.C,BOGOTÁ D.C. (CT),ARMA BLANCA,2010-01-01,FEMENINO,ADULTOS,1,Riesgo Alto
...,...,...,...,...,...,...,...,...
572074,VALLE DEL CAUCA,CARTAGO,SIN EMPLEO DE ARMAS,2023-02-28,FEMENINO,ADULTOS,1,Riesgo Alto
572075,VALLE DEL CAUCA,GUACARÍ,SIN EMPLEO DE ARMAS,2023-02-28,FEMENINO,ADULTOS,1,Riesgo Moderado
572076,VALLE DEL CAUCA,GUADALAJARA DE BUGA,SIN EMPLEO DE ARMAS,2023-02-28,FEMENINO,ADULTOS,1,Riesgo Moderado
572077,VALLE DEL CAUCA,PALMIRA,SIN EMPLEO DE ARMAS,2023-02-28,FEMENINO,ADULTOS,2,Riesgo Moderado


## Muestra del 20%

In [26]:
df['fecha_hecho'] = pd.to_datetime(df['fecha_hecho'])
df['year'] = df['fecha_hecho'].dt.year
casos_por_year = df.groupby('year')['cantidad'].sum()
casos_por_year

year
2010     22902
2011     26863
2012     32181
2013     32838
2014     47691
2015     72921
2016     82114
2017     83651
2018     79940
2019     92550
2020    109367
2021     91319
2022     94455
2023     13594
Name: cantidad, dtype: int64

In [27]:
df1 = df.groupby('year').sample(frac=0.2, random_state=1)
casos_por_year = df1['year'].value_counts().sort_index()
casos_por_year

year
2010     3050
2011     3430
2012     4118
2013     4394
2014     6003
2015     9013
2016     9925
2017    10045
2018     9520
2019    10479
2020    20494
2021    11080
2022    11237
2023     1629
Name: count, dtype: int64

In [28]:
df1 = df1.reset_index(drop=True)
# Guardamos el 80% restante de los datos
restante = df.drop(df1.index)
casos_por_year = restante['year'].value_counts().sort_index()
casos_por_year

year
2015     35621
2016     49624
2017     50226
2018     47598
2019     52397
2020    102469
2021     55401
2022     56183
2023      8143
Name: count, dtype: int64

In [29]:
restante = restante.reset_index(drop=True)

In [30]:
df1

,departamento,municipio,armas_medios,fecha_hecho,genero,grupo_etario,cantidad,cluster,year
0,CALDAS,LA DORADA,ARMA BLANCA,2010-06-01,FEMENINO,MENORES,1,Riesgo Moderado,2010
1,HUILA,NEIVA (CT),ARMA BLANCA,2010-11-29,FEMENINO,ADULTOS,2,Riesgo Moderado,2010
2,VALLE DEL CAUCA,CALI (CT),ARMA BLANCA,2010-05-28,FEMENINO,ADULTOS,1,Riesgo Alto,2010
3,BOYACA,DUITAMA,Unknown,2010-08-22,FEMENINO,ADULTOS,1,Riesgo Alto,2010
4,SANTAFE DE BOGOTA D.C,BOGOTÁ D.C. (CT),ARMA BLANCA,2010-11-25,MASCULINO,ADOLESCENTES,1,Riesgo Alto,2010
...,...,...,...,...,...,...,...,...,...
114412,AMAZONAS,LETICIA (CT),SIN EMPLEO DE ARMAS,2023-02-24,FEMENINO,ADULTOS,1,Riesgo Moderado,2023
114413,SANTANDER,BARRANCABERMEJA,ARMA BLANCA,2023-01-22,FEMENINO,ADULTOS,1,Riesgo Alto,2023
114414,CHOCO,QUIBDÓ (CT),ARMA BLANCA,2023-02-26,FEMENINO,ADULTOS,1,Riesgo Moderado,2023
114415,CAUCA,PAEZ,SIN EMPLEO DE ARMAS,2023-01-15,FEMENINO,ADULTOS,1,Riesgo Moderado,2023


In [31]:
df1.drop(columns=['departamento', 'fecha_hecho', 'year'], inplace=True)

## Entrenamiento y prueba

In [34]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Suponiendo que 'municipio', 'armas_medios', 'genero', 'grupo_etario', 'cluster' son categóricas
categorical_columns = ['municipio', 'armas_medios', 'genero', 'grupo_etario', 'cluster']

# Utilizando One-Hot Encoding para codificar las variables categóricas
encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = encoder.fit_transform(df1[categorical_columns])

# Convertir la matriz dispersa a DataFrame
X_encoded_df = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenar las variables codificadas con las variables numéricas (si las hay)
X = pd.concat([X_encoded_df, df1.drop(columns=categorical_columns)], axis=1)

# Definir la variable dependiente
y = df1['cantidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modelos de Regresión

El objetivo de la realización de modelos de regresión es predecir la cantidad de casos que pueden ocurrir en una zona teniendo en cuenta atributos como el tipo de arma, el género, el grupo etario y el tipo de riesgo de cada crimen.

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import joblib

# Definir los modelos
models = {
    'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'KNN': KNeighborsRegressor()
}

# Definir los hiperparámetros a ajustar para cada modelo
params = {
    'Lasso': {'alpha': [0.01, 0.1, 1, 10]},
    'Random Forest': {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20]},
    'XGBoost': {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]},
    'KNN' : {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']}
    }

# Realizar la búsqueda de hiperparámetros con validación cruzada para cada modelo
for name, model in models.items():
    print(f"Training {name}...")
    grid_search = GridSearchCV(model, params[name], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    print("Best parameters:", grid_search.best_params_)
    print("Best cross-validation score:", -grid_search.best_score_)
    print("Test set RMSE:", np.sqrt(-grid_search.score(X_test, y_test)))
    print()

    # Guardar el modelo
    joblib.dump(grid_search.best_estimator_, f"{name}_model.pkl")
    print(f"Trained {name} model saved as {name}_model.pkl\n")


Training Lasso...


c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted t

Best parameters: {'alpha': 0.01}
Best cross-validation score: 3.5948202816793364e-05


c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Test set RMSE: 0.006066584011539349

Trained Lasso model saved as Lasso_model.pkl

Training Random Forest...


...

Best parameters: {'max_depth': 5, 'n_estimators': 100}
Best cross-validation score: 2.9308632866666556e-11
Test set RMSE: 4.456253496136869e-06

Trained XGBoost model saved as XGBoost_model.pkl

Training KNN...


c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted t

Best parameters: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
Best cross-validation score: 0.005714681551580401


c:\Users\lisap\miniconda3\envs\ml_venv\lib\site-packages\sklearn\utils\validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Test set RMSE: 0.06938140312614335

Trained KNN model saved as KNN_model.pkl



## Evaluación de los modelos

In [57]:
LR_model = load('Lasso_model.pkl')
RF_model = load('Random Forest_model.pkl')
KNN_model = load('KNN_model.pkl')
XGB_model = load('XGBoost_model.pkl')

In [55]:
import warnings
warnings.filterwarnings("ignore")

# Importar las bibliotecas necesarias
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Definir la función para evaluar el modelo de regresión
def metricas(modelo, X_test, y_test, nombre):
    predicciones = modelo.predict(X_test)
    mse = mean_squared_error(y_test, predicciones)
    r2 = r2_score(y_test, predicciones)
    print("Métricas para el modelo", nombre)
    print("Mean Squared Error:", mse)
    print("R^2 Score:", r2)
    return predicciones

# Evaluar cada modelo de regresión
predicciones_LR = metricas(LR_model, X_test, y_test, "Regresión Logística")
predicciones_RF = metricas(RF_model, X_test, y_test, "Random Forest")
predicciones_KNN = metricas(KNN_model, X_test, y_test, "KNN")
predicciones_xgboost = metricas(XGB_model, X_test, y_test, "XGBoost")

Métricas para el modelo Regresión Logística
Mean Squared Error: 3.680344156906486e-05
R^2 Score: 0.999987090199123
Métricas para el modelo Random Forest
Mean Squared Error: 0.0
R^2 Score: 1.0
Métricas para el modelo KNN
Mean Squared Error: 0.004813779099752415
R^2 Score: 0.9983114370017047
Métricas para el modelo XGBoost
Mean Squared Error: 1.985819522183207e-11
R^2 Score: 0.9999999999930342


In [59]:
# Definir la función para evaluar el modelo de regresión
def metricas(modelo, X_test, y_test, nombre):
    predicciones = modelo.predict(X_test)
    mse = mean_squared_error(y_test, predicciones)
    r2 = r2_score(y_test, predicciones)
    return nombre, mse, r2

# Evaluar cada modelo de regresión
metricas_modelos = []

metricas_modelos.append(metricas(LR_model, X_test, y_test, "Regresión Logística"))
metricas_modelos.append(metricas(RF_model, X_test, y_test, "Random Forest"))
metricas_modelos.append(metricas(KNN_model, X_test, y_test, "KNN"))
metricas_modelos.append(metricas(XGB_model, X_test, y_test, "XGBoost"))

# Crear un DataFrame con las métricas obtenidas
metricas_df = pd.DataFrame(metricas_modelos, columns=['Modelo', 'MSE', 'R2'])

# Imprimir el DataFrame
print(metricas_df)

                Modelo           MSE        R2
0  Regresión Logística  3.680344e-05  0.999987
1        Random Forest  0.000000e+00  1.000000
2                  KNN  4.813779e-03  0.998311
3              XGBoost  1.985820e-11  1.000000


In [84]:
import plotly.graph_objs as go

# Crear el gráfico de barras para la columna R2
fig = go.Figure()

# Agregar un trace para cada modelo
for index, row in metricas_df.iterrows():
    fig.add_trace(go.Bar(
        x=[row['Modelo']],
        y=[row['R2']],
        name=row['Modelo']
    ))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='R^2 por Modelo',
    xaxis_title='Modelo',
    yaxis_title='R^2',
    barmode='group'
)

# Establecer el rango del eje y
fig.update_yaxes(range=[0.99, 1])

# Mostrar el gráfico
fig.show()

